---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [43]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import matplotlib as plt
#%matplotlib inline
from pdb import set_trace
from scipy.signal import argrelextrema

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [44]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [45]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    
    Un_sity= pd.read_table('university_towns.txt', header=None)
    tmp = ''
    deldubl= False
    Un_sity[1] = [0 for i in range(len(Un_sity))]
    for i in range(len(Un_sity)):
        if '[edit]' in Un_sity.iloc[i,0]:
             tmp= Un_sity.iloc[i,0].split('[')[0]        
        Un_sity.iloc[i,1] = tmp
        if '(' in Un_sity.iloc[i,0]:
            Un_sity.iloc[i,0] = Un_sity.iloc[i,0].split('(')[0]
    Un_sity.columns = ["RegionName","State"]
    Un_sity['RegionName'] = Un_sity['RegionName'].apply(lambda x: x.strip(' '))   
    Un_sity = (Un_sity[Un_sity.RegionName.str.contains('\[edit\]') == False]
               .reset_index(drop=True))
    col = Un_sity.columns.tolist()
    col = col[-1:] + col[:-1]
    Un_sity = Un_sity[col]
    return Un_sity

#get_list_of_university_towns()

In [46]:
gdplev = pd.read_excel('gdplev.xls',skiprows=5)

gdplev_year = gdplev.loc[:,'Unnamed: 0':'GDP in billions of chained 2009 dollars']
gdplev_year.columns = ['Date','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']
gdplev_year.dropna(axis=0, how='all',inplace=True)   
gdplev_year['Date'] = (gdplev_year['Date'].astype(str).apply(lambda x: pd.to_datetime(x, format='%Y.0'))
                       .apply(lambda x: x.year))


gdplev_quart = gdplev.loc[:,'Unnamed: 4':'GDP in billions of chained 2009 dollars.1']
gdplev_quart.dropna(axis=0, how='all',inplace=True)
gdplev_quart.columns = ['Date','GDP in billions of current dollars','GDP in billions of chained 2009 dollars']

gdplev_quart = gdplev_quart.set_index('Date')
gdplev_quart.drop(gdplev_quart.index[:-66], inplace=True)


qw_ind = list(gdplev_quart.index)
gdplev_quart_ress = gdplev_quart.iloc[-40:-15]
#gdplev_quart_ress.plot(y='GDP in billions of chained 2009 dollars', kind='line')

In [47]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''    
    x = argrelextrema(gdplev_quart['GDP in billions of chained 2009 dollars'].values,np.less)
    return gdplev_quart.index.values[x[0][2]]

#get_recession_start()

In [48]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    x = argrelextrema(gdplev_quart['GDP in billions of chained 2009 dollars'].values,np.greater)
    return gdplev_quart.index.values[x[0][4]]
#get_recession_end()

In [49]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    x = argrelextrema(gdplev_quart['GDP in billions of chained 2009 dollars'].values,np.less)
    return gdplev_quart.index.values[x[0][3]]
#get_recession_bottom()

In [50]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment descraiption, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    City_Zhvi= pd.read_csv('City_Zhvi_AllHomes.csv')
    City_Zhvi['State'] = City_Zhvi['State'].map(states)
    City_Zhvi = City_Zhvi.set_index(['State','RegionName']).iloc[:,-200:]
    qw_ind.append('2016q3')
    City_Zhvi_qw = pd.DataFrame(index=City_Zhvi.index,columns=qw_ind)

    City_t = City_Zhvi.T
    City_t.index = City_t.index.map(lambda x: x.split('-')[0])
    i=0
    for group, frame in City_t.groupby(level=0):
        if group != '2016':
            df1, df2, df3, df4 = np.array_split(frame, 4)
            City_Zhvi_qw[qw_ind[i]] =df1.mean(axis=0)
            City_Zhvi_qw[qw_ind[i+1]] =df2.mean(axis=0)
            City_Zhvi_qw[qw_ind[i+2]] =df3.mean(axis=0)
            City_Zhvi_qw[qw_ind[i+3]] =df4.mean(axis=0)
            i+=4

        else:
            df1, df2 = np.array_split(frame[:-2], 2)
            df3 = frame.iloc[-2:]

            City_Zhvi_qw[qw_ind[i]] =df1.mean(axis=0)
            City_Zhvi_qw[qw_ind[i+1]] =df2.mean(axis=0)
            City_Zhvi_qw[qw_ind[i+2]] =df3.mean(axis=0)

    
    return City_Zhvi_qw

#convert_housing_data_to_quarters()

In [51]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    price_ratio =  (lambda row : (row[oneq_befor] - row[get_recession_bottom()])/row[oneq_befor])
    def is_uni_town(row):
        #check if the town is a university towns or not.
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
    def is_better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town'
        else:
            return 'university town'
    
    def is_test_true():
        if p_val < 0.01:
            return True
        else:
            return False
    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)
    df = (convert_housing_data_to_quarters().copy()
          .loc[:,'2007q4':'2009q2']
          .reset_index())
    #oneq_befor = df.columns.get_loc(get_recession_start()) - 1
    oneq_befor = '2007q4'
    df['is_uni'] = df.apply(is_uni_town,axis=1)
    df['up&down'] = df.apply(price_ratio,axis=1)
    df['up&down'].to_csv('exit2.csv')    
    not_uni = df[df['is_uni']==0].loc[:,'up&down'].dropna()
    is_uni  = df[df['is_uni']==1].loc[:,'up&down'].dropna()
    p_val = list(ttest_ind(not_uni,is_uni))[1]
    return (is_test_true(),p_val,is_better())

run_ttest()

(True, 1.7900059546810142e-05, 'university town')